# Problem 1 - Josh

Read Shannon’s 1948 paper ’A Mathematical Theory of Communication’.  
Focus on pages 1-19 (up to Part II), the remaining part is more relevant for communication.
https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf

*Q: Summarize what you learned briefly (e.g. half a page).*

\<Summary\>

# Problem 2 - Jackson  

ICML is a top research conference in Machine learning. Scrape all the pdfs of all ICML 2017 papers from http://proceedings.mlr.press/v70/.
1. What are the top 10 common words in the ICML papers?
2. Let Zbe a randomly selected word in a randomly selected ICML paper. Estimate the entropy
of Z.
3. Synthesize a random paragraph using the marginal distribution over words.
4. (Extra credit) Synthesize a random paragraph using an n-gram model on words. Synthesize
a random paragraph using any model you want. Top five synthesized text paragraphs win
bonus (+30 points).

In [145]:
# Scraper
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from random import random
import pandas as pd
import numpy as np
import requests
import logging
import fitz
import os

def scrape(dump_folder, source):
    if dump_folder[-1] != '/':
        dump_folder += '/'
    #Create folder to dump into
    if not os.path.isdir(dump_folder):
        os.mkdir(dump_folder)

    #Set up logging
    f = open(f'{dump_folder}log.txt', 'w') #Open the file if its not already opened
    f.close()
    logging.basicConfig(level=logging.INFO, filename=f'{dump_folder}log.txt')
    
    #Get list of links
    html = requests.get(source)
    soup = bs(html.content, 'html.parser')
    links = soup.findAll('a')

    #Scrape all PDFs
    names = []
    for l in links:
        if l.decode_contents() == 'Download PDF' or l.decode_contents() == 'Supplementary PDF':
            src = l.get('href').replace('ı', 'i') # Fix small error in one of the scraped links
            fname = src[src.rindex('/')+1:]
            if fname in names:
                logging.CRITICAL(f'OVERWRITING FILE WITH NAME {fname}')
            names.append(fname)
            logging.info(f'Scraping pdf from {src} into {fname}')

            pdf = requests.get(src)
            with open(f'{dump_folder}{fname}', 'wb') as f:
                f.write(pdf.content)

def parse_pdf(fp):
    df = pd.DataFrame(columns=['word', 'prev'])
    pdf = fitz.open(fp)
    for page in pdf:
        _blocks = pdf[0].get_text_blocks()
        blocks = []
        for b in _blocks:
            if b[-1] != 1:
                blocks.append(b)

        #Create 2D array of words by block
        data = [x[4].replace('. ', ' . ').split() +['\end'] for x in blocks]
        if data == []:
            continue
        mlen = max([len(d) for d in data])
        data = [d + ['' for i in range(mlen - len(d) + 1)] for d in data]

        #Create array of previous words by block
        words = np.array(data)
        prev = np.roll(words, 1, axis=1)
        prev[:, 0] = '\start'

        words = words.flatten()
        prev = prev.flatten()

        df = pd.concat([df, pd.DataFrame(data = np.column_stack([words, prev]), columns=['word', 'prev'])])

    return df.drop(df[(df['word']=='') | (df['prev']=='')].index)


def parse_pdfs(dump_folder):
    df = pd.DataFrame(columns=['word', 'prev'])
    for fp in tqdm(os.listdir(dump_folder)):
        if not fp.endswith('pdf'):
            continue #Only process PDFs
        df = pd.concat([df, parse_pdf(dump_folder+fp)])

    df['count'] = 1
    df = pd.pivot_table(df, values='count', index=['prev', 'word'], aggfunc=np.sum)
    df = df.reset_index(level=1)
    df = df[(df.index.str.match(r'^[\\a-zA-Z\.]+$')) & (df['word'].str.match(r'^[\\a-zA-Z\.]+$'))]
    df['total'] = df.groupby(by='prev').transform('sum')['count'].astype(int)
    df['percent'] = df['count'] / df['total']
    df['cumsum'] = df.groupby(by='prev').transform('cumsum')['percent'].astype(float)

    return df

def predict(df, prev_word):
    """
    Given a previous word, and a marginal distribution dataframe, predict the word that follows it
    """
    p = random()
    words = df.loc[prev_word]
    if type(words) == pd.Series:
        return words['word']
    words = words[words['cumsum'] > p]
    return words.iloc[0]['word']
    

    

def synthesize_text(use_stored_data= True, df_fp='./distir.csv', save=True, source='http://proceedings.mlr.press/v70/', dump_folder = './scraped/', scrape_data=True):
    """If stored data exists, use it
    Otherwise scrape and process
    

    Then iteratively predict what word comes next"""

    # Scrape if necessary
    if not (use_stored_data and os.path.exists(df_fp)):
        if scrape_data:
            print('Scraping...', end='')
            scrape(dump_folder, source)
            print('Done!')
        df = parse_pdfs(dump_folder)
        
        #Save if wanted
        if save:
            df.to_csv(df_fp)
    else:
        df = pd.read_csv(df_fp, index_col=0)

    # Iteratively Synthesize Text
    words = ['\start']
    while words[-1] != '\end':
        words += [predict(df, words[-1])]
        
    return (' '.join(words[1:-1])).replace(' . ', '. ')

def top_words(df, n=10):
    return list(df.groupby(by='word')['count'].sum().sort_values(ascending=False).head(n).index)

def entropy(_df, word=None):
    if word is None:
        word = _df['word'].sample(1).values[0]

    df = _df[_df['word'] == word]

    return word, -1 * (np.sum(df['percent'] * np.log(df['percent'])/np.log(2)))

In [115]:
#top 10 words
df = pd.read_csv('./distir.csv')
top_words(df, 10)

['the', 'of', '.', 'a', 'to', '\\end', 'is', 'and', 'in', 'that']

In [152]:
entropy(df)

('implement', 0.1260897249795341)

In [155]:
synthesize_text()

'Z is still not have too sensitive to quadratically in the learnable embeddings separately by a good prediction As its evolution over binary vectors. These include the amount of the system. Exact solutions to the full history and useful and was naturally generate sparse generalized WNN by combining the neural architectures despite its performance of Thm. We note that leads to estimate the feasible region P. . Fix a typical case of the c.d.f. Correspondence'

# Takeaways and synthesized text

> C \\\\ S is the input perturbation of about individuals

> Z is still not have too sensitive to quadratically in the learnable embeddings separately by a good prediction As its evolution over binary vectors. These include the amount of the system. Exact solutions to the full history and useful and was naturally generate sparse generalized WNN by combining the neural architectures despite its performance of Thm. We note that leads to estimate the feasible region P. . Fix a typical case of the c.d.f. Correspondence

Above are examples of text that was produced using only the marginal distribution of words seen in the scraped PDFs, however it is not a very good approximation of human sentences.
This is due to the fact that words are not constructed in the context of the previously seen word, but rather all words in the sentence and paragraph prior to it.

An n-gram model takes the same concept but expands to be trained on the context of the n-previous words, with a higher n leading to much better text generation.
This still has its limitations however, as even a very large corpus does not have enough data to fully capture the range of possible human speech, so recurrent neural networks trained on tokenized text is a better next step.


# Problem 3 - Jhanvi

Continue building your toolbox on Kaggle. Work on submissions for the same competition
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/
1. What is the best Kaggle forum post that you found? Briefly describe what you learned from
it.
2. What is the best public leader board (LB) score you can achieve? Describe your approach.
3. Submit a model that is definitely overfitting and a model that is definitely underfitting.


Overfitting means that your training error is much smaller compared to your test error (and LB score).   
Underfitting means that your model is too simple and even the training error is very large (and so will the test error).  
You can experiment with depth of decision trees in random forests or XGBoost classifiers as the metric of complexity for your models, or any other family of models you want.